In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import re
import os # Import the os module for path manipulation

# Create the graph builder for applying graph algorithm

In [3]:
class GraphBuilder:
    """
    A class to parse UCLA nodes and nets files, build a NetworkX graph,
    and calculate graph centrality measures like Eigenvector Centrality and PageRank.
    """

    def __init__(self, nodes_file_content: str, nets_file_content: str):
        """
        Initializes the GraphBuilder with the content of UCLA nodes and nets files
        and builds the internal graph.

        Args:
            nodes_file_content (str): The entire content of the .nodes file as a string.
            nets_file_content (str): The entire content of the .nets file as a string.
        """
        self._nodes_file_content = nodes_file_content
        self._nets_file_content = nets_file_content
        self.graph = self._build_graph_from_ispd_data()
        print("GraphBuilder initialized and graph built.")

    def _parse_ucla_nodes(self) -> dict:
        """
        Parses the content of a UCLA nodes file.

        Returns:
            dict: A dictionary mapping node IDs (e.g., 'o0') to None,
                  just registering the node's existence.
        """
        nodes = {}
        lines = self._nodes_file_content.splitlines()
        for line in lines:
            line = line.strip()
            # Check if the line starts with 'o' and has at least 3 parts (node_id, x, y/width, height)
            # For simplicity, we only care about the node_id for graph construction.
            if line.startswith('o') and len(line.split()) >= 3:
                parts = line.split()
                node_id = parts[0]
                nodes[node_id] = None  # Just register the node's existence
        return nodes

    def _parse_ucla_nets(self) -> list:
        """
        Parses the content of a UCLA nets file to extract connections between nodes.

        Returns:
            list: A list of lists, where each inner list contains node IDs
                  connected by a single net.
        """
        nets_connections = []
        lines = self._nets_file_content.splitlines()
        current_net_pins = []
        in_net_block = False

        for line in lines:
            line = line.strip()
            if line.startswith('NetDegree :'):
                # If we were in a net block and collected pins, add them to connections
                if in_net_block and current_net_pins:
                    nets_connections.append(list(current_net_pins))
                current_net_pins = [] # Reset for the new net
                in_net_block = True
                # Example: "NetDegree : 4 n0"
                # We don't need 'n0' directly for graph edges, just the connected pins.
            elif line.startswith('o') and in_net_block:
                # Example: "o197239 I : -0.500000 -6.000000"
                parts = line.split()
                node_id = parts[0]
                current_net_pins.append(node_id)
            elif not line and in_net_block: # Empty line signals end of a net block (or almost)
                if current_net_pins:
                    nets_connections.append(list(current_net_pins))
                current_net_pins = []
                in_net_block = False
            elif not line and not in_net_block: # Multiple empty lines
                pass # Ignore
            # This condition handles cases where a net block might not be followed by an empty line
            # before the file ends or a new section begins.
            elif not line.startswith('#') and not line.startswith('Num') and in_net_block:
                # If we encounter a non-empty, non-comment, non-Num line while in a net block,
                # it's likely the end of the previous net.
                if current_net_pins:
                    nets_connections.append(list(current_net_pins))
                    current_net_pins = []
                in_net_block = False # Exit net block state

        # Append the last net's connections if loop ends within a net block
        if current_net_pins:
            nets_connections.append(list(current_net_pins))

        return nets_connections

    def _build_graph_from_ispd_data(self) -> nx.Graph:
        """
        Builds a NetworkX graph from ISPD2005 nodes and nets file content
        provided during initialization.

        Returns:
            nx.Graph: A graph representing the connections.
        """
        print("Parsing nodes file...")
        nodes_data = self._parse_ucla_nodes()
        print(f"Found {len(nodes_data)} unique nodes.")

        print("Parsing nets file...")
        nets_connections = self._parse_ucla_nets()
        print(f"Found {len(nets_connections)} nets.")

        G = nx.Graph()

        # Add all nodes first
        for node_id in nodes_data:
            G.add_node(node_id)

        print("Adding edges based on net connections (clique model)...")
        # For each net, create edges between all connected nodes (clique model)
        for net_pins in nets_connections:
            if len(net_pins) > 1: # Only add edges if there are at least two pins on a net
                # Add all possible pairs of nodes as edges for this net
                for i in range(len(net_pins)):
                    for j in range(i + 1, len(net_pins)):
                        G.add_edge(net_pins[i], net_pins[j])
        print(f"Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
        return G

    def calculate_eigenvector_centrality(self) -> list:
        """
        Calculates eigenvector centrality for each node in the built graph and returns
        the scores sorted in descending order.

        Returns:
            list: A list of tuples, where each tuple contains (node ID, eigenvector centrality score),
                  sorted by score in descending order. Returns an empty list on error.
        """
        print("Calculating Eigenvector Centrality...")
        try:
            # Use max_iter and tol for convergence control
            centrality = nx.eigenvector_centrality(self.graph, max_iter=1000, tol=1e-06)
            # Sort the items by their centrality score in descending order
            sorted_centrality = sorted(centrality.items(), key=lambda item: item[1], reverse=True)
            return sorted_centrality
        except nx.PowerIterationFailedConvergence:
            print("Warning: Power iteration failed to converge. Trying numpy version for robustness.")
            try:
                centrality = nx.eigenvector_centrality_numpy(self.graph)
                sorted_centrality = sorted(centrality.items(), key=lambda item: item[1], reverse=True)
                return sorted_centrality
            except Exception as e:
                print(f"Error with eigenvector_centrality_numpy: {e}")
                return []
        except Exception as e:
            print(f"An unexpected error occurred during centrality calculation: {e}")
            return []

    def calculate_pagerank(self, alpha: float = 0.85, max_iter: int = 1000, tol: float = 1e-06) -> list:
        """
        Calculates PageRank for each node in the built graph and returns
        the scores sorted in descending order.

        Args:
            alpha (float): Damping factor (default is 0.85).
            max_iter (int): Maximum number of iterations in power method.
            tol (float): Tolerance for convergence.

        Returns:
            list: A list of tuples, where each tuple contains (node ID, PageRank score),
                  sorted by score in descending order. Returns an empty list on error.
        """
        print(f"Calculating PageRank (alpha={alpha}, max_iter={max_iter}, tol={tol})...")
        try:
            # NetworkX's pagerank can handle both directed and undirected graphs.
            pagerank_scores = nx.pagerank(self.graph, alpha=alpha, max_iter=max_iter, tol=tol)
            # Sort the items by their PageRank score in descending order
            sorted_pagerank = sorted(pagerank_scores.items(), key=lambda item: item[1], reverse=True)
            return sorted_pagerank
        except nx.PowerIterationFailedConvergence:
            print("Warning: PageRank power iteration failed to converge. Try increasing max_iter or adjusting alpha/tol.")
            return []
        except Exception as e:
            print(f"An unexpected error occurred during PageRank calculation: {e}")
            return []

    def calculate_degree_centrality(self) -> list:
        """
        Calculates degree centrality for each node in the built graph and returns
        the scores sorted in descending order.

        Returns:
            list: A list of tuples, where each tuple contains (node ID, degree centrality score),
                  sorted by score in descending order. Returns an empty list on error.
        """
        print("Calculating Degree Centrality...")
        try:
            centrality = nx.degree_centrality(self.graph)
            # Sort the items by their centrality score in descending order
            sorted_centrality = sorted(centrality.items(), key=lambda item: item[1], reverse=True)
            return sorted_centrality
        except Exception as e:
            print(f"An unexpected error occurred during degree centrality calculation: {e}")
            return []


## Testing GraphBuilder class

In [4]:
benchmark_name = '/kaggle/input/ispd2005/ispd2005/adaptec1' # <--- CHANGE THIS TO YOUR DESIRED BENCHMARK
folder_name = benchmark_name.split("/")[-1]
nodes_file_path = os.path.join(benchmark_name, f"{folder_name}.nodes")
nets_file_path = os.path.join(benchmark_name, f'{folder_name}.nets')

print(f"Attempting to read nodes from: {nodes_file_path}")
print(f"Attempting to read nets from: {nets_file_path}")

ucla_nodes_content = ""
ucla_nets_content = ""

try:
    with open(nodes_file_path, 'r') as f:
        ucla_nodes_content = f.read()
    with open(nets_file_path, 'r') as f:
        ucla_nets_content = f.read()
except FileNotFoundError:
    print(f"Error: Make sure the folder '{benchmark_name}' exists and contains '{folder_name}.nodes' and '{folder_name}.nets'.")
    exit()
except Exception as e:
    print(f"An error occurred while reading files: {e}")
    exit()
graph_builder = GraphBuilder(ucla_nodes_content, ucla_nets_content)
graph = graph_builder.graph 

Attempting to read nodes from: /kaggle/input/ispd2005/ispd2005/adaptec1/adaptec1.nodes
Attempting to read nets from: /kaggle/input/ispd2005/ispd2005/adaptec1/adaptec1.nets
Parsing nodes file...
Found 211447 unique nodes.
Parsing nets file...
Found 221142 nets.
Adding edges based on net connections (clique model)...
Graph created with 211447 nodes and 3909436 edges.
GraphBuilder initialized and graph built.


In [5]:
if graph.number_of_nodes() > 0:
    # Call the updated method that returns sorted centrality
    eigen_centrality_sorted = graph_builder.calculate_eigenvector_centrality()

    if eigen_centrality_sorted:
        print(f"\nEigenvector Centrality for {benchmark_name} benchmark:")
        print("Top 5 Nodes by Eigenvector Centrality:")
        for node, score in eigen_centrality_sorted[:5]:
            print(f"Node {node}: {score:.6f}")

        print("\nBottom 5 Nodes by Eigenvector Centrality:")
        for node, score in eigen_centrality_sorted[-5:]:
            print(f"Node {node}: {score:.6f}")

        # Optional: Visualize a small subgraph for demonstration
        if graph.number_of_nodes() < 50:
            plt.figure(figsize=(10, 8))
            pos = nx.spring_layout(graph, k=0.15, iterations=20)
            # Recreate the dictionary for visualization if needed, or adjust node_sizes calculation
            eigen_centrality_dict = {node: score for node, score in eigen_centrality_sorted}
            node_sizes = [eigen_centrality_dict.get(node, 0) * 5000 for node in graph.nodes()]
            nx.draw_networkx(graph, pos, with_labels=True, node_size=node_sizes,
                             node_color='lightgreen', font_size=8, font_weight='bold',
                             edge_color='gray', alpha=0.7)
            plt.title(f"{benchmark_name} Graph with Node Size Proportional to Eigenvector Centrality")
            plt.show()
            print(f"\nGraph has {graph.number_of_nodes()} nodes. Skipping visualization for large graphs.")
    else:
        print("Could not calculate eigenvector centrality.")
else:
    print("No nodes found to build a graph.")

Calculating Eigenvector Centrality...

Eigenvector Centrality for /kaggle/input/ispd2005/ispd2005/adaptec1 benchmark:
Top 5 Nodes by Eigenvector Centrality:
Node o33313: 0.028305
Node o210929: 0.028304
Node o148188: 0.028301
Node o132201: 0.028301
Node o211444: 0.028301

Bottom 5 Nodes by Eigenvector Centrality:
Node o203095: 0.000000
Node o207695: 0.000000
Node o205421: 0.000000
Node o203023: 0.000000
Node o203021: 0.000000


In [6]:
if graph.number_of_nodes() > 0:
    # Call the updated method that returns sorted pagerank
    pagerank_scores_sorted = graph_builder.calculate_pagerank(alpha=0.85, max_iter=1000, tol=1e-06)

    if pagerank_scores_sorted:
        print(f"\nPageRank Scores for {benchmark_name} benchmark:")
        print("Top 5 Nodes by PageRank:")
        for node, score in pagerank_scores_sorted[:5]:
            print(f"Node {node}: {score:.6f}")

        print("\nBottom 5 Nodes by PageRank:")
        for node, score in pagerank_scores_sorted[-5:]:
            print(f"Node {node}: {score:.6f}")

        # Optional: Visualize a small subgraph for demonstration
        if graph.number_of_nodes() < 50:
            plt.figure(figsize=(10, 8))
            pos = nx.spring_layout(graph, k=0.15, iterations=20)
            # Recreate the dictionary for visualization if needed, or adjust node_sizes calculation
            pagerank_scores_dict = {node: score for node, score in pagerank_scores_sorted}
            node_sizes = [pagerank_scores_dict.get(node, 0) * 10000 for node in graph.nodes()] # Adjust multiplier for better visualization
            nx.draw_networkx(graph, pos, with_labels=True, node_size=node_sizes,
                             node_color='salmon', font_size=8, font_weight='bold',
                             edge_color='gray', alpha=0.7)
            plt.title(f"{benchmark_name} Graph with Node Size Proportional to PageRank")
            plt.show()
        else:
            print(f"\nGraph has {graph.number_of_nodes()} nodes. Skipping visualization for large graphs.")
    else:
        print("Could not calculate PageRank.")
else:
    print("No nodes found to build a graph.")

Calculating PageRank (alpha=0.85, max_iter=1000, tol=1e-06)...

PageRank Scores for /kaggle/input/ispd2005/ispd2005/adaptec1 benchmark:
Top 5 Nodes by PageRank:
Node o210925: 0.000198
Node o210924: 0.000152
Node o210908: 0.000137
Node o210909: 0.000134
Node o210906: 0.000133

Bottom 5 Nodes by PageRank:
Node o201232: 0.000001
Node o201236: 0.000001
Node o201228: 0.000001
Node o201245: 0.000001
Node o201230: 0.000001

Graph has 211447 nodes. Skipping visualization for large graphs.


In [7]:
if graph.number_of_nodes() > 0:
    degree_centrality_sorted = graph_builder.calculate_degree_centrality()

    if degree_centrality_sorted:
        print(f"\nDegree Centrality for {benchmark_name} benchmark:")
        print("Top 5 Nodes by Degree Centrality:")
        for node, score in degree_centrality_sorted[:5]:
            print(f"Node {node}: {score:.6f}")

        print("\nBottom 5 Nodes by Degree Centrality:")
        for node, score in degree_centrality_sorted[-5:]:
            print(f"Node {node}: {score:.6f}")

        # Optional: Visualize a small subgraph for demonstration
        if graph.number_of_nodes() < 50:
            plt.figure(figsize=(10, 8))
            pos = nx.spring_layout(graph, k=0.15, iterations=20)
            # Recreate the dictionary for visualization
            degree_centrality_dict = {node: score for node, score in degree_centrality_sorted}
            node_sizes = [degree_centrality_dict.get(node, 0) * 5000 for node in graph.nodes()] # Adjust multiplier as needed
            nx.draw_networkx(graph, pos, with_labels=True, node_size=node_sizes,
                             node_color='skyblue', font_size=8, font_weight='bold',
                             edge_color='gray', alpha=0.7)
            plt.title(f"{benchmark_name} Graph with Node Size Proportional to Degree Centrality")
            plt.show()
        else:
            print(f"\nGraph has {graph.number_of_nodes()} nodes. Skipping visualization for large graphs.")
    else:
        print("Could not calculate degree centrality.")
else:
    print("No nodes found to build a graph.")

Calculating Degree Centrality...

Degree Centrality for /kaggle/input/ispd2005/ispd2005/adaptec1 benchmark:
Top 5 Nodes by Degree Centrality:
Node o210925: 0.011180
Node o210924: 0.009700
Node o210929: 0.008366
Node o210919: 0.008196
Node o33313: 0.008040

Bottom 5 Nodes by Degree Centrality:
Node o210932: 0.000005
Node o210933: 0.000005
Node o210934: 0.000005
Node o210935: 0.000005
Node o211198: 0.000005

Graph has 211447 nodes. Skipping visualization for large graphs.


# Create new dataset from the GraphBuilder

In [8]:
from pathlib import Path
from typing import List, Dict, Tuple
import shutil
class BenchmarkProcessor:
    """
    A class to process all ISPD2005 benchmarks in a given directory,
    rearranging the nodes and saving the new benchmark suite.
    """
    def __init__(self, root_dir: str, method_name: str):
        """
        Initializes the BenchmarkProcessor.

        Args:
            root_dir (str): The path to the root directory containing the benchmarks.
            method_name (str): A string describing the rearrangement method. This
                               will be used in the new folder names. Must be
                               one of 'eigenvector', 'pagerank', or 'degree'.
        """
        self.root_dir = Path(root_dir)
        self.method_name = method_name.lower()
        self.output_dir = self.root_dir.parent / f"{self.root_dir.name}_{self.method_name}"
        self.centrality_methods = {
            'eigenvector': 'calculate_eigenvector_centrality',
            'pagerank': 'calculate_pagerank',
            'degree': 'calculate_degree_centrality',
        }
        if self.method_name not in self.centrality_methods:
            raise ValueError(f"Invalid method name: {method_name}. Must be one of {list(self.centrality_methods.keys())}.")

    def _process_single_benchmark(self, benchmark_path: Path):
        """
        Processes a single benchmark folder:
        - Reads .nodes and .nets files.
        - Calls the GraphBuilder class to calculate centrality.
        - Creates a new folder for the modified benchmark.
        - Writes the new .nodes file and copies the others.

        Args:
            benchmark_path (Path): The Path object for the benchmark directory.
        """
        benchmark_name = benchmark_path.name
        print(f"\nProcessing benchmark: {benchmark_name}")

        nodes_file = benchmark_path / f"{benchmark_name}.nodes"
        nets_file = benchmark_path / f"{benchmark_name}.nets"
        
        # Check for required files
        if not nodes_file.exists() or not nets_file.exists():
            print(f"Skipping {benchmark_name}: .nodes or .nets file not found.")
            return

        # 1. Read file content
        try:
            with open(nodes_file, 'r') as f:
                nodes_content = f.read()
            with open(nets_file, 'r') as f:
                nets_content = f.read()
        except IOError as e:
            print(f"Error reading files for {benchmark_name}: {e}")
            return

        # 2. Use GraphBuilder to get centrality scores
        try:
            graph_builder = GraphBuilder(nodes_content, nets_content)
            method_to_call = getattr(graph_builder, self.centrality_methods[self.method_name])
            centrality_scores = method_to_call()
        except Exception as e:
            print(f"Error in GraphBuilder centrality calculation for {benchmark_name}: {e}")
            return

        # 3. Create a new .nodes file content
        new_nodes_content = self._generate_new_nodes_file(nodes_content, centrality_scores)

        # 4. Create a new directory for the output
        new_benchmark_name = f"{benchmark_name}_{self.method_name}"
        new_benchmark_path = self.output_dir / new_benchmark_name
        
        if new_benchmark_path.exists():
            shutil.rmtree(new_benchmark_path)
        new_benchmark_path.mkdir(parents=True)
        print(f"Created new benchmark folder: {new_benchmark_path}")
        
        # 5. Write the new .nodes file
        new_nodes_file_path = new_benchmark_path / f"{benchmark_name}.nodes"
        with open(new_nodes_file_path, 'w') as f:
            f.write('\n'.join(new_nodes_content))
        print(f"Saved new .nodes file to: {new_nodes_file_path}")

        # 6. Copy all other files
        files_to_copy = [f for f in os.listdir(benchmark_path) if not f.endswith('.nodes')]
        for file_name in files_to_copy:
            shutil.copy(benchmark_path / file_name, new_benchmark_path / file_name)
        print(f"Copied {len(files_to_copy)} other files.")

    def _generate_new_nodes_file(self, original_nodes_content: str, scores: List[Tuple[str, float]]) -> List[str]:
        """
        Generates the content for the new .nodes file, including the original
        data and a comment block with the calculated centrality scores.
        """
        original_lines = original_nodes_content.splitlines()
        header = []
        node_lines = []
        for line in original_lines:
            line = line.strip()
            if line.startswith(('NumNodes', 'NumTerminals')):
                header.append(line)
            else:
                node_lines.append(line)

        modified_nodes_data = header
        modified_nodes_data.extend(node_lines)
        
        modified_nodes_data.append("\n# --- Centrality Scores ---")
        modified_nodes_data.append(f"# Scores are calculated using the '{self.method_name}' method.")
        modified_nodes_data.append("# Format: node_id, score\n")

        for node_id, score in scores:
            modified_nodes_data.append(f"# {node_id} {score}")

        return modified_nodes_data

    def process_all_benchmarks(self):
        """
        Runs through all valid benchmark folders in the root directory
        and processes each one.
        """
        if not self.root_dir.is_dir():
            print(f"Error: Directory not found at {self.root_dir}")
            return

        # Ensure the output directory is ready
        if self.output_dir.exists():
            print(f"Output directory '{self.output_dir}' already exists. Overwriting its contents.")
            shutil.rmtree(self.output_dir)
        self.output_dir.mkdir(parents=True)

        print(f"Starting to process benchmarks from {self.root_dir}")
        for item in self.root_dir.iterdir():
            if item.is_dir():
                # Assumes each sub-directory is a benchmark
                self._process_single_benchmark(item)
        
        print("\nAll benchmarks processed successfully!")

In [9]:
# !cp -r /kaggle/input/ispd2005/ispd2005 /kaggle/working/ispd2005

In [10]:
# import shutil
# import os

# dir_path = '/kaggle/working/ispd2005_degree'  # Replace with your directory

# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
#     print(f"Deleted: {dir_path}")
# else:
#     print(f"Directory does not exist: {dir_path}")


In [ ]:
benchmark_dir = "/kaggle/working/ispd2005"
method = "degree"
try:
    processor = BenchmarkProcessor(root_dir=benchmark_dir, method_name=method)
    processor.process_all_benchmarks()
    print(f"\nNew benchmark files are located in the '{processor.output_dir}' folder.")
except ValueError as e:
    print(e)



Starting to process benchmarks from /kaggle/working/ispd2005

Processing benchmark: bigblue4
Parsing nodes file...
Found 2177353 unique nodes.
Parsing nets file...
Found 2229886 nets.
Adding edges based on net connections (clique model)...
Graph created with 2177353 nodes and 162088542 edges.
GraphBuilder initialized and graph built.
Calculating Degree Centrality...
Created new benchmark folder: /kaggle/working/ispd2005_degree/bigblue4_degree
Saved new .nodes file to: /kaggle/working/ispd2005_degree/bigblue4_degree/bigblue4.nodes
Copied 9 other files.

Processing benchmark: adaptec4
Parsing nodes file...
Found 496045 unique nodes.
Parsing nets file...
Found 515951 nets.
Adding edges based on net connections (clique model)...
Graph created with 496045 nodes and 13093493 edges.
GraphBuilder initialized and graph built.
Calculating Degree Centrality...
Created new benchmark folder: /kaggle/working/ispd2005_degree/adaptec4_degree
Saved new .nodes file to: /kaggle/working/ispd2005_degree/ad

# Testing BBO RS (In testing)

In [1]:
!git clone https://github.com/lamda-bbo/WireMask-BBO.git


Cloning into 'WireMask-BBO'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 58 (delta 0), reused 0 (delta 0), pack-reused 57 (from 1)
Receiving objects: 100% (58/58), 142.85 KiB | 3.97 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!pip install -r /kaggle/working/WireMask-BBO/requirements.txt
!pip install pyunpack
!pip install gpytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 17.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 85.3 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement opencv_python==4.1.2.30 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.1

In [4]:
import sys
sys.path.append('/kaggle/working/WireMask-BBO')

In [5]:
from place_db import PlaceDB
from utils import random_guiding, greedy_placer_with_init_coordinate, write_final_placement, rank_macros
from common import grid_setting, my_inf
import random
import time
import csv
import os

def BBO_RS(dataset, seed, stop_round):
    """
    Runs a greedy placement algorithm with random initial guiding.

    Args:
        dataset (str): The name of the dataset to use.
        seed (int): The random seed for reproducibility.
        stop_round (int): The number of rounds to run the placement for.
    """
    random.seed(seed)
    placedb = PlaceDB(dataset)
    
    hpwl_save_dir = "result/Random/curve/"
    placement_save_dir = "result/Random/placement/"
    node_id_ls = rank_macros(placedb)

    if not os.path.exists(hpwl_save_dir):
        os.makedirs(hpwl_save_dir)
    if not os.path.exists(placement_save_dir):
        os.makedirs(placement_save_dir)

    hpwl_save_dir += "{}_seed_{}.csv".format(dataset, seed)
    placement_save_dir += "{}_seed_{}.csv".format(dataset, seed)
    
    hpwl_save_file = open(hpwl_save_dir, "a+")
    hpwl_writer = csv.writer(hpwl_save_file)

    grid_num = grid_setting[dataset]["grid_num"]
    grid_size = grid_setting[dataset]["grid_size"]

    best_hpwl = my_inf
    best_placed_macro = None
    
    for _ in range(stop_round):
        print("init")
        place_record = random_guiding(node_id_ls, placedb, grid_num, grid_size)
        placed_macros, hpwl = greedy_placer_with_init_coordinate(node_id_ls, placedb, grid_num, grid_size, place_record)
        
        if hpwl < best_hpwl:
            best_hpwl = hpwl
            best_placed_macro = placed_macros
            write_final_placement(best_placed_macro, placement_save_dir)
        
        hpwl_writer.writerow([hpwl, time.time(), "init"])
        hpwl_save_file.flush()
    
    hpwl_save_file.close()



In [6]:
BBO_RS(dataset="adaptec1", seed=42, stop_round=100)

AssertionError: 